In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import random
import time
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
LENGTH=100
random.seed(5)
state_M=np.zeros(4,dtype=int)
state_J=0
MS_J1=np.array([1,2,4])
T_J1=np.array([8,16,12])
check_J1=np.zeros(3,dtype=int)
gantt_chart = pd.DataFrame({
    'M1':np.zeros(LENGTH),
    'M2':np.zeros(LENGTH),
    'M3':np.zeros(LENGTH),
    'M4':np.zeros(LENGTH)},
    index = [np.arange(1,LENGTH+1)],dtype=int)

In [ ]:
for t in np.arange(1,LENGTH+1):
    if np.any(state_M==0) :
        if state_J == 0:
            for i in np.arange(3):
                if state_J==0:        
                    if check_J1[i]==0:
                        if i==0 or (i>0  and check_J1[i-1]==1) :
                            m=MS_J1[i]-1
                            if state_M[m]==0:
                                gantt_chart.iloc[t-1:t-1+T_J1[i],m]=1
                                state_M[m] += T_J1[i]                       
                                state_J += T_J1[i]
                                check_J1[i] = 1                           
    state_J -= 1
    state_M[state_M !=0] -= 1 
    if np.all(check_J1==1):
        break
    
print(gantt_chart)
print(state_J)
print(state_M)
print(check_J1)
print(t)
                    
raw_data=gantt_chart
np.savetxt(fname='d:/pysrc/data/gantt_chart_simple.csv',X=raw_data,fmt='%0d',delimiter=',')

In [ ]:
import itertools
import math

list_arrange_J1 = list(itertools.permutations(np.arange(0,5)))
print(list_arrange_J1)
for a in list_arrange_J1:
    for j in a :
        print(j)
    print('')






In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 设置参数
mean1, std_dev1 = 0, 1   # 第一峰值的均值和标准差
mean2, std_dev2 = 5, 1.5 # 第二峰值的均值和标准差
size = 1000  # 样本总数
weight1, weight2 = 0.5, 0.5  # 两个分布的权重（权重和需为1）

# 生成随机数
data1 = np.random.normal(mean1, std_dev1, int(size * weight1))
data2 = np.random.normal(mean2, std_dev2, int(size * weight2))

# 合并数据
data = np.concatenate([data1, data2])

# 绘制直方图
plt.hist(data, bins=30, density=True, alpha=0.6, color='blue', edgecolor='black')

# 标题和标签
plt.title("Bimodal Normal Distribution")
plt.xlabel("Value")
plt.ylabel("Density")
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 创建示例 DataFrame
index = ['m1', 'm2', 'm3', 'm4']
columns = [str(i) for i in range(1, 101)]
data = np.random.choice([0, 1], size=(4, 100), p=[0.7, 0.3])  # 随机生成0或1
df = pd.DataFrame(data, index=index, columns=columns)

# 绘制图像
plt.figure(figsize=(12, 6))
plt.imshow(df, cmap='Greys', aspect='auto', interpolation='nearest')

# 设置轴标签
plt.xticks(ticks=np.arange(len(df.columns)), labels=df.columns, fontsize=8, rotation=90)
plt.yticks(ticks=np.arange(len(df.index)), labels=df.index, fontsize=10)
plt.xlabel('Columns (1 to 100)', fontsize=12)
plt.ylabel('Index (m1 to m4)', fontsize=12)

# 添加网格线（可选）
plt.grid(False)

# 显示图像
plt.colorbar(label='0 or 1', orientation='vertical')  # 可选：添加颜色说明
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import numpy as np
# 10行5列のランダムな数値データを作成
data = np.random.rand(10, 5)
df = pd.DataFrame(data, columns=['A', 'B', 'C', 'D', 'E'])
print("元のDataFrame:")
print(df)
# インデックスを削除したNumpy配列を作成
array_without_index = df.to_numpy()
print("\nNumpy配列:")
print(array_without_index)

In [ ]:
import numpy as np

# 创建一个示例二维数组
array = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])

# 提取列数
num_columns = array.shape[1]

print(num_columns)  # 输出: 列数: 3


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

# 订单数据
processing_times = np.array([
    [15, 10, 10, 15, 11, 14, -1, -1],
    [14, 14, 14, 10, -1, -1, -1, -1],
    [9, 16, 9, 9, 15, -1, -1, -1],
    [9, 10, 13, 10, -1, -1, -1, -1],
    [16, 8, 10, 10, 9, 8, 9, 9],
    [14, 10, 10, 8, 8, -1, -1, -1],
    [9, 9, 9, 16, 10, 8, -1, -1],
    [16, 16, 14, 8, -1, -1, -1, -1]
])

machine_matrix = np.array([
    [1, 3, 1, 4, 3, 2, -1, -1],
    [1, 2, 3, 2, -1, -1, -1, -1],
    [1, 4, 2, 1, 4, -1, -1, -1],
    [3, 2, 4, 2, -1, -1, -1, -1],
    [2, 1, 2, 3, 2, 1, 3, 1],
    [4, 1, 2, 3, 1, -1, -1, -1],
    [1, 2, 4, 1, 4, 3, -1, -1],
    [2, 3, 2, 4, -1, -1, -1, -1]
])

num_jobs = processing_times.shape[0]
num_machines = 4
num_tasks = np.count_nonzero(processing_times != -1)

# **智能初始化**
def create_initial_solution():
    job_order = []
    for j in range(num_jobs):
        for t in range(len(processing_times[j])):
            if processing_times[j][t] != -1:
                job_order.append((j, t))
    return job_order

# **适应度计算**
def evaluate(individual):
    job_completion = {j: 0 for j in range(num_jobs)}
    machine_completion = {m: 0 for m in range(1, num_machines + 1)}
    process_start_time = {}

    for job, task in individual:
        process_time = processing_times[job][task]
        machine = machine_matrix[job][task]

        start_time = max(job_completion[job], machine_completion[machine])
        end_time = start_time + process_time
        process_start_time[(job, task)] = start_time

        job_completion[job] = end_time
        machine_completion[machine] = end_time

    return (max(machine_completion.values()),), process_start_time

# **遗传算法**
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_initial_solution)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def run_ga():
    pop = toolbox.population(n=200)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    log = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.1, ngen=200, stats=stats, halloffame=hof, verbose=True)
    best_schedule = hof[0]

    return best_schedule, log

best_schedule, log = run_ga()

# **处理日志数据 (折线图数据)**
gen = log[1]  # 取日志数据
iterations = list(range(len(gen.select("min"))))
best_makespan = gen.select("min")

# **绘制甘特图**
def plot_gantt(best_schedule):
    _, start_times = evaluate(best_schedule)

    fig, ax = plt.subplots(figsize=(10, 6))
    colors = plt.cm.get_cmap("tab10", num_jobs)  # 生成不同 Job 颜色

    for (job, task), start_time in start_times.items():
        machine = machine_matrix[job][task]
        duration = processing_times[job][task]
        ax.barh(y=machine, width=duration, left=start_time, color=colors(job), edgecolor="black", label=f"J{job+1}-P{task+1}")

    ax.set_xlabel("时间 (min)")
    ax.set_ylabel("机器编号")
    ax.set_yticks(range(1, num_machines + 1))
    ax.set_yticklabels([f"M{i}" for i in range(1, num_machines + 1)])
    ax.set_title("生产调度甘特图")
    plt.show()

# **绘制进化折线图**
def plot_evolution_chart(iterations, best_makespan):
    plt.figure(figsize=(10, 5))
    plt.plot(iterations, best_makespan, marker='o', linestyle='-', color='b', label="最优 Makespan")
    plt.xlabel("迭代次数")
    plt.ylabel("最优完成时间 (makespan)")
    plt.title("GA 进化过程中 makespan 变化")
    plt.legend()
    plt.grid()
    plt.show()

# **调用绘图函数**
plot_gantt(best_schedule)
plot_evolution_chart(iterations, best_makespan)
